# Implementation of a new rPPG method

## Part 1 : Notebook for the training of the 3D-CNN model

This jupyter notebook file is composed of different sections allowing to train a Deep Learning neural network to estimate the heart rate of a patient with a simple video sequence.  With this jupyter notebook file, the results of the model can also be validated.

![General Schema](./img/general_scheme.JPG)

The objective is to generate a functional model in view of integrating it into the pyVHR Framework. ([Link](https://ieeexplore.ieee.org/document/9272290)) ([GitHub](https://github.com/phuselab/pyVHR))

This file is based on the implementation described in the following article :
Frédéric Bousefsaf, Alain Pruski, Choubeila Maaoui, 3D convolutional neural networks for remote pulse rate measurement and mapping from facial video, Applied Sciences, vol. 9, n° 20, 4364 (2019). ([Link](https://www.mdpi.com/2076-3417/9/20/4364)) ([GitHub](https://github.com/frederic-bousefsaf/ippg-3dcnn))

## Importing libraries

Previously , you have to install theses python librairies :
* tensorflow
* matplotlib
* scipy
* numpy
* opencv-python

In [1]:
import os
#RUN ON CPU 
os.environ['CUDA_VISIBLE_DEVICES'] = '-1'

#Tensorflow/KERAS
import tensorflow as tf
from tensorflow.python.keras.models import Sequential
from tensorflow.python.keras.callbacks import ModelCheckpoint
from tensorflow.python.keras.models import model_from_json
from tensorflow.python.keras.layers import ZeroPadding3D, Dense, Activation,Conv3D,MaxPooling3D,AveragePooling3D,Flatten,Dropout
from tensorflow.python.keras.utils import np_utils
from tensorflow.python.keras.models import model_from_json

# Numpy / Matplotlib / OpenCV / Scipy
import numpy as np
import matplotlib.pyplot as plt
import cv2
import scipy.io

## Setting of input data

Input Data = IMAGE_WIDTH * IMAGE_HEIGHT * LENGTH_VIDEOS

Units = (X pixels) * (Y pixels) * (N frames)

In [2]:
LENGTH_VIDEO = 60 #frames
IMAGE_WIDTH = 25 #pixels
IMAGE_HEIGHT = 25 #pixels
IMAGE_CHANNELS = 1 #1 or 2 or 3

#Time notion
SAMPLING = 1 / 30 #30 Hz
t = np.linspace(0, LENGTH_VIDEO * SAMPLING - SAMPLING, LENGTH_VIDEO)

## Setting of output data

A class represents an interval \[X,Y\] BPM frequencies.

For example, Class 1 can set the rating between 55 and 57.5 BPM.

In [3]:
# Available Outputs
HEART_RATES = np.linspace(55, 240, 75)
NB_CLASSES = len(HEART_RATES)

# prepare labels and label categories
labels = np.zeros(NB_CLASSES + 1)

## Setting of training session

In [11]:
# CONSTANTS
NB_VIDEOS_BY_CLASS_TRAIN = 1
NB_VIDEOS_BY_CLASS_VALIDATION = 100

EPOCHS = 1
CONTINUE_TRAINING = True #load or not a old trained model
MIXED_DATA = False

RESULTS_PATH = "./model"

## Binarization of classes

Class 1 : (1, 0, 0, 0, 0, ... , 0)

Class 2 : (0, 1, 0, 0, 0, ... , 0)

...

Class n : (0, 0, 0, 0, 0, ... , 1)


In [5]:
for i in range(NB_CLASSES + 1):
    labels[i] = i
labels_cat = np_utils.to_categorical(labels)

## Definition of the 3D CNN model

![Model](./img/model_structure.jpg)

In [6]:
# DEFINE MODEL
model = Sequential()

#feature extraction
model.add(Conv3D(filters=32, kernel_size=(LENGTH_VIDEO-2,IMAGE_HEIGHT-5,IMAGE_WIDTH-5), input_shape=(LENGTH_VIDEO, IMAGE_HEIGHT, IMAGE_WIDTH, IMAGE_CHANNELS)))
model.add(MaxPooling3D(pool_size=(2,2,2)))
model.add(Activation('relu'))
model.add(Dropout(0.2))

model.add(Flatten())

#Classification
model.add(Dense(512, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(NB_CLASSES + 1, activation='softmax'))

model.summary()
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv3d (Conv3D)              (None, 3, 6, 6, 32)       742432    
_________________________________________________________________
max_pooling3d (MaxPooling3D) (None, 1, 3, 3, 32)       0         
_________________________________________________________________
activation (Activation)      (None, 1, 3, 3, 32)       0         
_________________________________________________________________
dropout (Dropout)            (None, 1, 3, 3, 32)       0         
_________________________________________________________________
flatten (Flatten)            (None, 288)               0         
_________________________________________________________________
dense (Dense)                (None, 512)               147968    
_________________________________________________________________
dropout_1 (Dropout)          (None, 512)               0

# Process for artificial data creation

For more information, the process is described with more detail in the article cited in the introduction. However, here is a summary diagram:

![Data Generation](./img/data_generation.jpg)


Original version of the implementation of this process : [GitHub](https://github.com/frederic-bousefsaf/ippg-3dcnn/tree/master/train)

## Trend generation function for artificial data creation

In [7]:
def generate_trend(length, order, min, max, offset):

    if (order==1):   # linear
        tend = np.linspace(min, max, length)

    elif (order==2): # quadratic
        if (offset==0):
            tend = np.linspace(0, 1, length)
            tend = tend*tend
            tend = tend-min
            tend = max*tend/np.max(tend)

        else:
            tend = tend = np.linspace(-0.5, 0.5, length)
            tend = tend*tend
            tend = tend-min
            tend = 0.5*max*tend/np.max(tend)

    elif (order==3): # cubic
        if (offset==0):
            tend = np.linspace(0, 1, length)
            tend = tend*tend*tend
            tend = tend-min
            tend = max*tend/np.max(tend)

        else:
            tend = tend = np.linspace(-0.5, 0.5, length)
            tend = tend*tend*tend
            tend = tend-min
            tend = 0.5*max*tend/np.max(tend)
    return tend

## Trend generation parameters for artificial data creation

In [8]:
# Tendencies (linear, 2nd order, 3rd order)
TENDANCIES_MIN = (-3,-1,-1)
TENDANCIES_MAX = (3,1,1)
TENDANCIES_ORDER = (1,2,3)

# coefficients for the fitted-ppg method
a0 = 0.440240602542388
a1 = -0.334501803331783
b1 = -0.198990393984879
a2 = -0.050159136439220
b2 = 0.099347477830878
w = 2 * np.pi

## Train Data generation

In [12]:
xtrain = np.zeros(shape=((NB_CLASSES + 1) * NB_VIDEOS_BY_CLASS_TRAIN, LENGTH_VIDEO, IMAGE_HEIGHT, IMAGE_WIDTH, IMAGE_CHANNELS))
ytrain = np.zeros(shape=((NB_CLASSES + 1) * NB_VIDEOS_BY_CLASS_TRAIN, NB_CLASSES + 1))

c = 0

for i_freq in range(len(HEART_RATES)):

    for i_videos in range(NB_VIDEOS_BY_CLASS_TRAIN):

        t2 = t + (np.random.randint(low=0, high=33) * SAMPLING)   # phase
        signal = a0 + a1 * np.cos(t2 * w * HEART_RATES[i_freq] / 60) + b1 * np.sin(t2 * w * HEART_RATES[i_freq] / 60) + a2 * np.cos(2 * t2 * w * HEART_RATES[i_freq] / 60) + b2 * np.sin(2 * t2 * w * HEART_RATES[i_freq] / 60)
        signal = signal - np.min(signal)
        signal = signal / np.max(signal)

        r = np.random.randint(low=0, high=len(TENDANCIES_MAX))
        trend = generate_trend(len(t), TENDANCIES_ORDER[r], 0, np.random.uniform(low=TENDANCIES_MIN[r], high=TENDANCIES_MAX[r]), np.random.randint(low=0, high=2))

        signal = np.expand_dims(signal + trend, 1)
        signal = signal - np.min(signal)

        img = np.tile(signal, (IMAGE_WIDTH, 1, IMAGE_HEIGHT))
        img = np.transpose(img, axes=(0,2,1))

        img = img / (IMAGE_HEIGHT * IMAGE_WIDTH)
        
        amplitude = np.random.uniform(low=1.5, high=4)
        noise_energy = amplitude * 0.25 * np.random.uniform(low=1, high=10) / 100

        for j in range(0, LENGTH_VIDEO):
            temp = 255 * ((amplitude * img[:,:,j]) + np.random.normal(size=(IMAGE_HEIGHT, IMAGE_WIDTH), loc=0.5, scale=0.25) * noise_energy)
            temp[temp < 0] = 0 
            xtrain[c,j,:,:,0] = temp.astype('uint8') / 255.0

        xtrain[c] = xtrain[c] - np.mean(xtrain[c])
        ytrain[c] = labels_cat[i_freq]

        c = c + 1


# constant image noise (gaussian distribution)
for i_videos in range(NB_VIDEOS_BY_CLASS_TRAIN):
    r = np.random.randint(low=0, high=len(TENDANCIES_MAX))
    trend = generate_trend(len(t), TENDANCIES_ORDER[r], 0, np.random.uniform(low=TENDANCIES_MIN[r], high=TENDANCIES_MAX[r]), np.random.randint(low=0, high=2))

    # add a tendancy on noise
    signal = np.expand_dims(trend, 1)
    img = np.tile(signal, (IMAGE_WIDTH, 1, IMAGE_HEIGHT)) / (IMAGE_HEIGHT * IMAGE_WIDTH)
    img = np.expand_dims(np.transpose(img, axes=(1,0,2)), 3)

    xtrain[c] = np.expand_dims(np.random.normal(size=(LENGTH_VIDEO, IMAGE_HEIGHT, IMAGE_WIDTH)) / 50, 3) + img
    xtrain[c] = xtrain[c] - np.mean(xtrain[c])
    ytrain[c] = labels_cat[NB_CLASSES]
    c = c + 1

print('Train data generation done.')


Train data generation done.


# Training phase

## Start a training session + save

In [10]:
# load model
if (CONTINUE_TRAINING == True):
    model = model_from_json(open(f'{RESULTS_PATH}/model_conv3D.json').read())
    model.load_weights(f'{RESULTS_PATH}/weights_conv3D.h5')
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    
if(MIXED_DATA == True):
    data = np.load('./dataSplited.npz')
    xtrain = np.concatenate((xtrain, data['a']), axis=0)
    ytrain =  np.concatenate((ytrain, data['b']), axis=0)

    indices = np.arange(xtrain.shape[0])
    np.random.shuffle(indices)
    xtrain = xtrain[indices]
    ytrain = ytrain[indices]
    
#start training
model.fit(xtrain, ytrain, epochs = EPOCHS, batch_size=256, verbose=1)

#save data
model_json = model.to_json()
open(f'{RESULTS_PATH}/model_conv3D.json', 'w').write(model_json)
model.save_weights(f'{RESULTS_PATH}/weights_conv3D.h5', overwrite=True)
print('A new model has been saved!\n')

Epoch 1/200
30/30 [==============================] - 173s 6s/step - loss: 1.4092 - accuracy: 0.7283
Epoch 2/200
30/30 [==============================] - 176s 6s/step - loss: 0.8306 - accuracy: 0.7749
Epoch 3/200
30/30 [==============================] - 180s 6s/step - loss: 0.5980 - accuracy: 0.8125
Epoch 4/200
30/30 [==============================] - 176s 6s/step - loss: 0.4777 - accuracy: 0.8299
Epoch 5/200
30/30 [==============================] - 174s 6s/step - loss: 0.4104 - accuracy: 0.8533
Epoch 6/200
30/30 [==============================] - 172s 6s/step - loss: 0.4008 - accuracy: 0.8622
Epoch 7/200
30/30 [==============================] - 172s 6s/step - loss: 0.3399 - accuracy: 0.8758
Epoch 8/200
30/30 [==============================] - 172s 6s/step - loss: 0.2887 - accuracy: 0.8929
Epoch 9/200
30/30 [==============================] - 172s 6s/step - loss: 0.2798 - accuracy: 0.8976
Epoch 10/200
30/30 [==============================] - 172s 6s/step - loss: 0.2571 - accuracy: 0.9053

30/30 [==============================] - 181s 6s/step - loss: 0.0286 - accuracy: 0.9907
Epoch 163/200
30/30 [==============================] - 172s 6s/step - loss: 0.0257 - accuracy: 0.9914
Epoch 164/200
30/30 [==============================] - 171s 6s/step - loss: 0.0268 - accuracy: 0.9924
Epoch 165/200
30/30 [==============================] - 172s 6s/step - loss: 0.0327 - accuracy: 0.9899
Epoch 166/200
30/30 [==============================] - 171s 6s/step - loss: 0.0334 - accuracy: 0.9913
Epoch 167/200
30/30 [==============================] - 172s 6s/step - loss: 0.0247 - accuracy: 0.9911
Epoch 168/200
30/30 [==============================] - 172s 6s/step - loss: 0.0260 - accuracy: 0.9920
Epoch 169/200
30/30 [==============================] - 171s 6s/step - loss: 0.0279 - accuracy: 0.9909
Epoch 170/200
30/30 [==============================] - 172s 6s/step - loss: 0.0267 - accuracy: 0.9918
Epoch 171/200
30/30 [==============================] - 171s 6s/step - loss: 0.0297 - accuracy: 0

## Validation data generation

In [9]:
data = {}

# 2.  GENERATE VALIDATION DATA
xvalidation = np.zeros(shape=((NB_CLASSES + 1) * NB_VIDEOS_BY_CLASS_VALIDATION, LENGTH_VIDEO, IMAGE_HEIGHT, IMAGE_WIDTH, IMAGE_CHANNELS))
yvalidation = np.zeros(shape=((NB_CLASSES + 1) * NB_VIDEOS_BY_CLASS_VALIDATION, NB_CLASSES + 1))

c = 0

# for each frequency
for i_freq in range(len(HEART_RATES)):

    for i_videos in range(NB_VIDEOS_BY_CLASS_VALIDATION):

        t2 = t + (np.random.randint(low=0, high=33) * SAMPLING)   # phase. 33 corresponds to a full phase shift for HR=55 bpm
        signal = a0 + a1 * np.cos(t2 * w * HEART_RATES[i_freq] / 60) + b1 * np.sin(t2 * w * HEART_RATES[i_freq] / 60) + a2 * np.cos(2 * t2 * w * HEART_RATES[i_freq] / 60) + b2 * np.sin(2 * t2 * w * HEART_RATES[i_freq] / 60)
        signal = signal - np.min(signal)
        signal = signal / np.max(signal)

        r = np.random.randint(low=0, high=len(TENDANCIES_MAX))
        trend = generate_trend(len(t), TENDANCIES_ORDER[r], 0, np.random.uniform(low=TENDANCIES_MIN[r], high=TENDANCIES_MAX[r]), np.random.randint(low=0, high=2))

        signal = np.expand_dims(signal + trend, 1)
        signal = signal - np.min(signal)

        img = np.tile(signal, (IMAGE_WIDTH, 1, IMAGE_HEIGHT))
        img = np.transpose(img, axes=(0,2,1))

        img = img / (IMAGE_HEIGHT * IMAGE_WIDTH)
        
        amplitude = np.random.uniform(low=1.5, high=4)
        noise_energy = amplitude * 0.25 * np.random.uniform(low=1, high=10) / 100

        for j in range(0, LENGTH_VIDEO):
            temp = 255 * ((amplitude * img[:,:,j]) + np.random.normal(size=(IMAGE_HEIGHT, IMAGE_WIDTH), loc=0.5, scale=0.25) * noise_energy)
            temp[temp < 0] = 0 
            xvalidation[c,j,:,:,0] = temp.astype('uint8') / 255.0

        xvalidation[c] = xvalidation[c] - np.mean(xvalidation[c])
        yvalidation[c] = labels_cat[i_freq]

        c = c + 1


# constant image noise (gaussian distribution)
for i_videos in range(NB_VIDEOS_BY_CLASS_VALIDATION):
    r = np.random.randint(low=0, high=len(TENDANCIES_MAX))
    trend = generate_trend(len(t), TENDANCIES_ORDER[r], 0, np.random.uniform(low=TENDANCIES_MIN[r], high=TENDANCIES_MAX[r]), np.random.randint(low=0, high=2))

    # add a tendancy on noise
    signal = np.expand_dims(trend, 1)
    img = np.tile(signal, (IMAGE_WIDTH, 1, IMAGE_HEIGHT)) / (IMAGE_HEIGHT * IMAGE_WIDTH)
    img = np.expand_dims(np.transpose(img, axes=(1,0,2)), 3)

    xvalidation[c] = np.expand_dims(np.random.normal(size=(LENGTH_VIDEO, IMAGE_HEIGHT, IMAGE_WIDTH)) / 50, 3) + img
    xvalidation[c] = xvalidation[c] - np.mean(xvalidation[c])
    yvalidation[c] = labels_cat[NB_CLASSES]
    c = c + 1

print('Validation data generation done')

Validation data generation done


# Testing phase

## Start a validation session

In [10]:
model = model_from_json(open(f'{RESULTS_PATH}/model_conv3D.json').read())
model.load_weights(f'{RESULTS_PATH}/weights_conv3D.h5')
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

if(MIXED_DATA == True):
    data = np.load('./dataSplited.npz')
    xvalidation = np.concatenate((xvalidation, data['c']), axis=0)
    yvalidation =  np.concatenate((yvalidation, data['d']), axis=0)

    indices = np.arange(xvalidation.shape[0])
    np.random.shuffle(indices)
    xvalidation = xvalidation[indices]
    yvalidation = yvalidation[indices]
    
model.evaluate(xvalidation, yvalidation, verbose=1)


238/238 [==============================] - 59s 246ms/step - loss: 1.2349 - accuracy: 0.7538


[1.2348835468292236, 0.753815770149231]